# Chat 对象使用示例

这个示例展示了如何使用 Chat 对象进行对话管理，包括：
- 从 .env 文件加载配置
- 基本对话功能
- 异步对话
- 对话历史管理
- 错误处理
- 高级功能

## 1. 环境准备和导入模块

In [1]:
import os
from datetime import datetime

# 导入 chattool 相关模块
from chattool.core.chattype import Chat
from chattool.core.config import OpenAIConfig

## 2. 创建 Chat 对象

使用环境变量自动配置

In [2]:
# 方法1: 使用默认配置（自动从环境变量读取）
chat = Chat()
print("✅ Chat 对象创建成功（使用环境变量配置）")

# 方法2: 使用显式配置
config = OpenAIConfig(
    api_key=os.getenv('OPENAI_API_KEY'),
    api_base=os.getenv('OPENAI_API_BASE'),
    model=os.getenv('OPENAI_API_MODEL', 'gpt-3.5-turbo'),
    temperature=0.7,
    max_tokens=1000
)
chat_with_config = Chat(config=config)
print("✅ Chat 对象创建成功（使用显式配置）")

✅ Chat 对象创建成功（使用环境变量配置）
✅ Chat 对象创建成功（使用显式配置）


## 3. 基本对话示例

In [3]:
# 设置系统提示
chat.system("你是一个友善的AI助手，请用简洁明了的方式回答问题。")

# 进行第一轮对话
print("=== 第一轮对话 ===")
response1 = chat.user("你好！请简单介绍一下自己。").get_response()
print(f"用户: 你好！请简单介绍一下自己。")
print(f"助手: {response1.content}")
print(f"Token使用: {response1.total_tokens}")
print()

=== 第一轮对话 ===
用户: 你好！请简单介绍一下自己。
助手: 你好！我是一个友善的AI助手，可以帮助你回答问题，提供信息和解决问题。有什么可以帮到你的吗？
Token使用: 99



In [4]:
# 继续对话
print("=== 第二轮对话 ===")
response2 = chat.user("能告诉我今天是星期几吗？").get_response()
print(f"用户: 能告诉我今天是星期几吗？")
print(f"助手: {response2.content}")
print(f"Token使用: {response2.total_tokens}")
print()

=== 第二轮对话 ===
用户: 能告诉我今天是星期几吗？
助手: 当然！今天是星期二。有什么我可以帮助你的吗？
Token使用: 147



## 4. 查看对话历史

In [5]:
# 查看完整对话历史
print("=== 对话历史 ===")
for i, message in enumerate(chat.chat_log):
    role = message['role']
    content = message['content'][:100] + '...' if len(message['content']) > 100 else message['content']
    print(f"{i+1}. [{role.upper()}]: {content}")

print(f"\n总消息数: {len(chat)}")
print(f"最后一条消息: {chat.last_message}")

=== 对话历史 ===
1. [SYSTEM]: 你是一个友善的AI助手，请用简洁明了的方式回答问题。
2. [USER]: 你好！请简单介绍一下自己。
3. [ASSISTANT]: 你好！我是一个友善的AI助手，可以帮助你回答问题，提供信息和解决问题。有什么可以帮到你的吗？
4. [USER]: 能告诉我今天是星期几吗？
5. [ASSISTANT]: 当然！今天是星期二。有什么我可以帮助你的吗？

总消息数: 5
最后一条消息: 当然！今天是星期二。有什么我可以帮助你的吗？


## 5. 便捷问答方法

In [6]:
# 使用 ask 方法进行快速问答
print("=== 便捷问答 ===")
answer = chat.ask("请用一句话解释什么是人工智能？")
print(f"问题: 请用一句话解释什么是人工智能？")
print(f"回答: {answer}")

=== 便捷问答 ===
问题: 请用一句话解释什么是人工智能？
回答: 人工智能是一种模拟人类智慧和学习能力的技术。


## 6. 异步对话示例

In [7]:
# 异步对话函数
async def async_chat_example():
    print("=== 异步对话示例 ===")
    
    # 创建新的对话实例
    async_chat = Chat()
    async_chat.system("你是一个编程助手，请提供简洁的编程建议。")
    
    # 异步获取响应
    response = await async_chat.ask_async("Python中如何创建虚拟环境？")
    print(f"问题: Python中如何创建虚拟环境？")
    print(f"回答: {response}")
    
    return async_chat

# 运行异步示例
async_chat = await async_chat_example()

=== 异步对话示例 ===
问题: Python中如何创建虚拟环境？
回答: 你可以使用 `venv` 模块来创建Python虚拟环境。下面是一些简单的步骤：

1. 打开命令行，进入你想要创建虚拟环境的目录。
2. 运行以下命令来创建一个名为 `myenv` 的虚拟环境：

```
python -m venv myenv
```

3. 激活虚拟环境：

在 Windows 上：

```
myenv\Scripts\activate
```

在 macOS 和 Linux 上：

```
source myenv/bin/activate
```

激活后，你会看到终端提示已进入虚拟环境。在虚拟环境中安装的包不会影响全局Python环境。


## 7. 批量异步处理

In [8]:
# 批量处理多个问题
async def batch_questions():
    print("=== 批量异步处理 ===")
    
    questions = [
        "什么是机器学习？",
        "什么是深度学习？",
        "什么是神经网络？"
    ]
    
    # 为每个问题创建独立的对话
    tasks = []
    for question in questions:
        temp_chat = Chat()
        temp_chat.system("请用一句话简洁回答问题。")
        task = temp_chat.ask_async(question)
        tasks.append((question, task))
    
    # 并发执行所有任务
    for i, (question, task) in enumerate(tasks):
        answer = await task
        print(f"{i+1}. 问题: {question}")
        print(f"   回答: {answer}")
        print()

await batch_questions()

=== 批量异步处理 ===
1. 问题: 什么是机器学习？
   回答: 机器学习是一种人工智能的分支，利用数据和算法使计算机学习和改进性能。

2. 问题: 什么是深度学习？
   回答: 深度学习是一种基于人工神经网络模型、具有多层结构的机器学习方法。

3. 问题: 什么是神经网络？
   回答: 神经网络是一种模拟人脑神经元之间相互连接和通信方式的人工智能模型。



## 8. 对话管理功能

In [9]:
# 对话复制
print("=== 对话管理 ===")
copied_chat = chat.copy()
print(f"原对话消息数: {len(chat)}")
print(f"复制对话消息数: {len(copied_chat)}")

# 在复制的对话中尝试不同的问题
copied_response = copied_chat.ask("换个话题，你能推荐一本好书吗？")
print(f"\n复制对话中的新回答: {copied_response}")

# 删除最后一条消息
removed_message = chat.pop()
print(f"\n删除的消息: {removed_message}")
print(f"删除后消息数: {len(chat)}")

=== 对话管理 ===
原对话消息数: 7
复制对话消息数: 7

复制对话中的新回答: 当然！我推荐《活着》这本书，它是余华的代表作品，描写了中国农村普通人的生活经历，充满了对生命的思考和感悟。非常值得一读！

删除的消息: {'role': 'assistant', 'content': '人工智能是一种模拟人类智慧和学习能力的技术。'}
删除后消息数: 6


## 9. 调试和监控

In [10]:
# 调试信息
print("=== 调试和监控 ===")

# 打印对话历史
print("对话历史:")
chat.print_log()

# 获取最后一次响应的详细信息
if chat.last_response:
    print("\n最后一次响应详情:")
    print(f"ID: {chat.last_response.id}")
    print(f"模型: {chat.last_response.model}")
    print(f"创建时间: {chat.last_response.created}")
    print(f"完成原因: {chat.last_response.finish_reason}")
    print(f"Token统计: {chat.last_response.usage}")

=== 调试和监控 ===
对话历史:

--------------------------------------------------
SYSTEM
--------------------------------------------------
你是一个友善的AI助手，请用简洁明了的方式回答问题。

--------------------------------------------------
USER
--------------------------------------------------
你好！请简单介绍一下自己。

--------------------------------------------------
ASSISTANT
--------------------------------------------------
你好！我是一个友善的AI助手，可以帮助你回答问题，提供信息和解决问题。有什么可以帮到你的吗？

--------------------------------------------------
USER
--------------------------------------------------
能告诉我今天是星期几吗？

--------------------------------------------------
ASSISTANT
--------------------------------------------------
当然！今天是星期二。有什么我可以帮助你的吗？

--------------------------------------------------
USER
--------------------------------------------------
请用一句话解释什么是人工智能？

最后一次响应详情:
ID: chatcmpl-CR1HXbzaMfiDF8isi314lCtHNtwW7
模型: gpt-3.5-turbo-0125
创建时间: 1760555819
完成原因: stop
Token统计: {'prompt_tokens': 173, 'completion_tokens': 28, 'total_tokens': 20

# 10. 异步对话示例

In [11]:
import asyncio

chat = Chat().user('hi')
async def stream_example():
    # 使用异步流式响应
    accumulated_content = ""
    
    async for response in chat.get_response_stream_async():
        # 获取增量内容
        if response.delta_content:
            accumulated_content += response.delta_content
            print(response.delta_content, end='', flush=True)
        
        # 检查是否完成
        if response.finish_reason == 'stop':
            print("\n\n流式响应完成!")
            break
    
    print(f"完整内容长度: {len(accumulated_content)}")

# 运行异步函数
asyncio.run(stream_example())


Hello! How can I assist you today?

流式响应完成!
完整内容长度: 34
